# Part I. ETL Pipeline for Pre-Processing Raw Data

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder of event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-24-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra data modelling. 

## Now one can use the CSV file titled <font color=red>event_datafile_new.csv</font>.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [5]:
# The code below makes a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creation of a Keyspace

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS my_keyspace
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
#Set KEYSPACE to the keyspace specified above

session.set_keyspace("my_keyspace")

# Data Modelling

Now we start with the data modelling in Apache Cassandra. It is crucial that we model (in a NoSQL context) the data based on the way users want to interact with the data, i.e. the query they might use. Best practice is to use a single table for each query. Hence, we will model the data for each query separately. Each query answers a question about the data. The following question will be considered:   



### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1:
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Since this question about the data is conditioned to the session number and the items in the session, we will partition the data with respect to these columns. That means, we choose a composite primary key which consists of session id and the items in session column. By doing that, the data gets distributed across the nodes based on this primary key. Every query which include the session id and the items in session column in a WHERE condition, can then look up the data just from a single node. 

### Definition of the cql statement which creates the table

In [9]:
str_create_table = """
CREATE TABLE IF NOT EXISTS song_playlist_item_session
    (sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
PRIMARY KEY (sessionId, itemInSession))
"""

session.execute(str_create_table)                    

### Insert data in the table define above from csv file

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_item_session (sessionId, itemInSession, artist, song, length) VALUES"
        query = query + "(%s, %s, %s, %s, %s)"
        # look up the relevant columns based on the csv file header
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### SELECT to verify that the data have been inserted

In [11]:
## Execute the query that answers the first question

query = """
    SELECT
        artist,
        song,
        length
    FROM
        song_playlist_item_session
    WHERE
        sessionId=338 AND itemInSession=4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question, we choose a partition key which consists of the user Id and the session Id since these columns appear in the WHERE clause. But they do not uniquely identify a row in the dataset. Hence, we define the primary key as consisting of the partition key and the clustering column item in session. The clustering column determine the order in which the data is sorted within the partition (i.e. in our case, the data will be sorted by the item in session column as required).

### Definition of the cql statement which creates the table

In [12]:
str_create_table = """
CREATE TABLE IF NOT EXISTS  song_playlist_session
    (userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
PRIMARY KEY ((userId, sessionId), itemInSession))
"""

session.execute(str_create_table)

### Insert data

In [13]:
str_insert = """
INSERT INTO song_playlist_session
    (userId, sessionId, itemInSession, artist, song, firstName, lastName)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(str_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### SELECT to verify that the data have been inserted 

In [14]:
# Query that answers the second question

query_2 = """
    SELECT
        artist,
        song,
        firstname,
        lastname
    FROM
        song_playlist_session
    WHERE 
        userId=10 AND sessionId=182
    ORDER BY
        itemInSession DESC"""

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


# Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this case, we chose the primary equals the partition key consisting of the column’s song and user Id. It is necessary to include also the user id, since the song column (which would just be needed from a partitioning perspective) does not uniquely identify a record.

### Definition of the cql statement which creates the table

In [15]:
str_create_table = """
CREATE TABLE IF NOT EXISTS song_playlist_library
    (song text,
    userId int,
    firstName text,
    lastName text,
PRIMARY KEY (song, userId))
"""

session.execute(str_create_table)                    

### Insert data

In [17]:
str_insert = """
INSERT INTO song_playlist_library (song, userId, firstName, lastName)
VALUES (%s, %s, %s, %s)
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(str_insert, (line[9], int(line[10]), line[1], line[4]))

### SELECT to verify that the data have been inserted

In [18]:
query_3 = """
    SELECT
        firstname,
        lastname
    FROM
        song_playlist_library
    WHERE
        song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions
session.execute("drop table song_playlist_item_session")
session.execute("drop table song_playlist_session")
session.execute("drop table song_playlist_library")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()